Assign a Chromosome class

In [58]:
import pandas as pd
import re
import os

distance = 5 #目標基因的上下n個
score = 2 #出現次數

class Chromosome: #創立一個chromosome 的物件
    count = 0 #總共有染色體數量
    specieslist = []     #宣告空的物種列表
    def __init__(self, filename):  #initilize filename
        self.filename = filename
        Chromosome.count+=1


        def get_data(filename):  #取得網路上下載的csv檔案
            data = pd.read_csv("data\\syntenic\\" + filename, skiprows=[0]) #讀取
            data["Gene symbol"] = data["Gene symbol"].apply(lambda x: x.upper()) #將symbol統一為大寫（cacng1 -> CACNG1）
            try:
                data = data.drop('Name', axis=1).reset_index().drop("index", axis = 1).dropna() #刪除csv資料内的 'name' column
                return data.dropna()
            except: #若原本就沒有 'name' column的情況
                return data

        def get_species(filename):  #利用檔案名字來取得物種名稱 （檔案名字格式一定要是物種名含空格，接著_後面為染色體名字）
            species = re.search("[A-Za-z\s]+", filename) #正則表達取得符合字母以及底綫的格式的字串
            return species.group(0) #因為符合格式的字串被存在一個串列，因此我們只需要group(0)，第一個的物件

        def get_chromosome_location(filename):  #利用檔案名字來取得染色體的名字 (Anolis carolinensis_scaffold NW_003338810.1.CSV -> scaffold NW_003338810.1)
            temp = re.sub(".CSV", "", filename)  #去除.csv
            location = re.search("[_]([A-Za-z0-9\s_.]+)", temp) #正則表達取得符合字母以及底綫的格式的字串
            return location.group(1) #為了把re格式的[_]去掉 (_scaffold NW_003338810.1 -> scaffold NW_003338810.1)

        def get_targetgenes(data):  #傳入data，取得 cacng 或 tmem114 或 235 target 基因放進一個串列内
            targetgeneslist = []  
            for gene in data["Gene symbol"]:
                if "CACNG" in gene.upper():
                    targetgeneslist.append(gene)
                elif "TMEM114" in gene.upper():
                    targetgeneslist.append(gene)
                elif "TMEM235" in gene.upper():
                    targetgeneslist.append(gene)
            return targetgeneslist


        def get_neighbour_genes(targetgenes,data,n): #取得鄰近n個基因的函數
            neighbourindex = []  #宣告一個新的鄰近基因的引數 (利用引數來取得基因的目的是因為引述可以排列)
            neighbourgene = []
            for index, gene in enumerate(data["Gene symbol"]):  #a loop to save neighbourgene index
                if gene in targetgenes:
                    for i in range (index-n, index+n+1):
                        neighbourindex.append(i)
            neighbourindex = list(dict.fromkeys(neighbourindex)) #刪掉重叠的基因引數 (因為可能cacng1，4，5很近，他們鄰近的會重複出現)
            for i in neighbourindex: #利用鄰近基因的引數來取得鄰近基因的名字
                try: #防止索引的值超出data的範圍 (如果cacng出現在最前端或最後端)
                    neighbourgene.append(data["Gene symbol"][i])
                except: #超出的話會出錯，要除錯
                    pass
            return neighbourgene

        self.data = get_data(self.filename)
        self.species = get_species(self.filename)
        self.location = get_chromosome_location(self.filename)
        self.targetgenes = get_targetgenes(self.data)
        self.neighbour = get_neighbour_genes(self.targetgenes,self.data,distance)
        self.textx = None
        self.texty = None

        if self.species not in Chromosome.specieslist:
            Chromosome.specieslist.append(self.species)

    def set_genes(self,gene):
        self.genes = gene


Initialize class

In [59]:
filelist = os.listdir("data\\syntenic")  #get file list
for index, file in enumerate(filelist):  #a loop to initialize class chromosome
    exec(f"c{index} = Chromosome(file)")  #initialize chromosome
    exec('print(f"c{index}: ", end = "")')  #print initialized class name
    exec(f"print(c{index}.species, c{index}.location, c{index}.neighbour)") #print species and location
    print()

c0: Anolis carolinensis C2 ['PGS1', 'LOC103277795', 'SOCS3', 'LOC103277796', 'LOC100564571', 'TMEM235', 'BIRC5', 'AFMID', 'TK1', 'SYNGR2', 'TMC8']

c1: Anolis carolinensis C5 ['MYH9', 'LOC103278846', 'TXN2', 'FOXRED2', 'EIF3D', 'CACNG2', 'IFT27', 'PVALB', 'NCF4', 'CSF2RB', 'TEX33']

c2: Anolis carolinensis scaffold NW_003338797.1 ['CACNG5', 'CACNG4', 'CACNG1', 'HELZ', 'PSMD12', 'PITPNC1', 'LOC107983219', 'NOL11']

c3: Anolis carolinensis scaffold NW_003338810.1 ['RBFOX1', 'MIR5411', 'LOC107983238', 'LOC107983239', 'TMEM114', 'METTL22', 'ABAT', 'TMEM186', 'PMM2', 'CARHSP1']

c4: Anolis carolinensis scaffold NW_003338876.1 ['CCDC114', 'MYADM', 'LOC100557974', 'PRKCG', 'LOC103280215', 'CACNG6', 'CACNG8', 'CACNG7', 'LOC100560082', 'RPS9', 'LMTK3', 'LOC100560478', 'LOC100560675']

c5: Callorhinchus milii scaffold NW_024704761.1 ['AXIN2', 'CEP112', 'APOH', 'LOC103175619', 'TRNAA-GGC', '(CACNG5-LIKE) LOC103175617', '(CACNG4-LIKE) LOC103175616', '(CACNG1-LIKE) LOC103175615', 'HELZ', 'PSMD12', 

In [60]:
speciesdict = {}        #############create a speciesdict
for i in Chromosome.specieslist:
    speciesdict[i] = []

for index in range(Chromosome.count):  #a loop to initialize class chromosome
    exec(f"speciesdict[c{index}.species].append('c{index}')")

speciesdict

{'Anolis carolinensis': ['c0', 'c1', 'c2', 'c3', 'c4'],
 'Callorhinchus milii': ['c5', 'c6', 'c7'],
 'Canis lupus familiaris': ['c8', 'c9', 'c10', 'c11'],
 'Chrysemys picta': ['c12', 'c13', 'c14', 'c15', 'c16', 'c17'],
 'Danio rerio': ['c18', 'c19', 'c20', 'c21', 'c22', 'c23'],
 'Gallus gallus': ['c24', 'c25', 'c26'],
 'Homo sapiens': ['c27', 'c28', 'c29', 'c30'],
 'Mus musculus': ['c31', 'c32', 'c33', 'c34'],
 'Oreochromis niloticus': ['c35', 'c36', 'c37', 'c38', 'c39'],
 'Petromyzon marinus': ['c40', 'c41', 'c42', 'c43', 'c44'],
 'Xenopus tropicalis': ['c45', 'c46', 'c47', 'c48']}

Create dataframe

In [61]:
df = pd.DataFrame()

for index, file in enumerate(filelist):  #add column name species and location to class.data (Dataframe)
     exec(f"df{index} = c{index}.data")  #assign df 
     exec(f"df{index}.insert(0,'Species', c{index}.species)")  #insert species
     exec(f"df{index}.insert(1,'Location', c{index}.location)")  #insert location
     exec(f"df = df.append(df{index}, ignore_index=True)")

df['Gene symbol'] = df['Gene symbol'].str.upper()  #change gene symbol to upper letter

Set distance

In [62]:
distance = distance  #distance as neighbour
neighbourindex = []  #a new list to save index of neighbour genes
for index, gene in enumerate(df["Gene symbol"]):  #a loop to save neighbourgene index
    if ("CACNG") in gene:
        for i in range (index-distance, index+distance+1):
            neighbourindex.append(i)
    elif ("TMEM114") in gene:
        for i in range (index-distance, index+distance+1):
            neighbourindex.append(i)
    elif ("TMEM235") in gene:
        for i in range (index-distance, index+distance+1):
            neighbourindex.append(i)

neighbourindex = sorted(list(set(neighbourindex)))  #remove duplicates
newdf = df.iloc[neighbourindex, :-1]  #a new data by neighbourindex and remove name column


In [63]:
scoredict = {}  #a dictionary to store score
for gene in newdf["Gene symbol"]:  #loop to calculate score
    if "CACNG" in gene:
        scoredict[gene] = 100
    elif "TMEM114" in gene:
        scoredict[gene] = 100
    elif "TMEM235" in gene:
        scoredict[gene] = 100
    else:
        try:
            scoredict[gene]
            scoredict[gene].append(1)  #add score
        except:
            scoredict[gene] = [1]  #build score

for gene in scoredict:  #append list item to score
    if type(scoredict[gene]) == list:
        scoredict[gene] = len(scoredict[gene])


Set a minimum score

In [64]:
score = score  ###set a minimum score
genelist = []  #gene that higher than score
for gene in scoredict: 
    if scoredict[gene] >= score:
        genelist.append(gene)


for index in range (Chromosome.count):  #remove each neighbourgene that not in genelist(score lower than n)
    templist = []
    exec(f"""
for neighbourgene in c{index}.neighbour:
    if neighbourgene.upper() in genelist:
        templist.append(neighbourgene.upper())""")  #append score that is higher than n to a templist
    exec(f"c{index}.set_genes(templist)")  #set Chromosome.gene as templist
    exec(f'print("c{index}: ", end="")')  #print 
    exec(f"print(c{index}.species, c{index}.location, c{index}.genes)") #print species and location
    print()

c0: Anolis carolinensis C2 ['PGS1', 'SOCS3', 'TMEM235', 'BIRC5', 'AFMID', 'TK1', 'SYNGR2']

c1: Anolis carolinensis C5 ['MYH9', 'TXN2', 'FOXRED2', 'EIF3D', 'CACNG2', 'IFT27', 'PVALB', 'NCF4', 'CSF2RB', 'TEX33']

c2: Anolis carolinensis scaffold NW_003338797.1 ['CACNG5', 'CACNG4', 'CACNG1', 'HELZ', 'PSMD12', 'PITPNC1', 'NOL11']

c3: Anolis carolinensis scaffold NW_003338810.1 ['RBFOX1', 'TMEM114', 'METTL22', 'ABAT', 'TMEM186', 'PMM2', 'CARHSP1']

c4: Anolis carolinensis scaffold NW_003338876.1 ['MYADM', 'PRKCG', 'CACNG6', 'CACNG8', 'CACNG7']

c5: Callorhinchus milii scaffold NW_024704761.1 ['AXIN2', 'CEP112', 'APOH', '(CACNG5-LIKE) LOC103175617', '(CACNG4-LIKE) LOC103175616', '(CACNG1-LIKE) LOC103175615', 'HELZ', 'PSMD12', 'NOL11', 'TK1', 'AFMID', 'BIRC5A', 'TMEM235B', 'THA1']

c6: Callorhinchus milii scaffold_NW_024704757.1 ['PRKCBA', '(LOW CACNG2-LIKE) LOC103186647', 'RBBP6', 'SI:CH211-14K19.8', 'NDUFAB1B', 'CARHSP1', 'PMM2', 'TMEM186', 'ABAT', 'METTL22', '(TMEM114-LIKE) LOC103178830'

Initialize draw location

In [65]:
buildxy = """
initialpoint = 300
tempx = initialpoint
tempy = initialpoint
step = 80
species = c0.species
c0.textx = initialpoint
c0.texty = initialpoint
for index in range(1,Chromosome.count):
    exec(f'''
if c{index}.species == species:
    c{index}.textx = tempx
    c{index}.texty = c{index-1}.texty + step*len(c{index-1}.genes) + 300
else:
    tempx += 1000
    species = c{index}.species
    c{index}.texty = initialpoint
    c{index}.textx = tempx

#print(c{index}.textx,c{index}.texty)''') 
"""

exec(buildxy)

Draw Image (png)

In [66]:
draw = """
from PIL import Image, ImageDraw, ImageFont

exec(f"newImage = Image.new('RGBA', (c{Chromosome.count-1}.textx + 800,10000), 'White')")
drawObj = ImageDraw.Draw(newImage)

fontInfo1 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf', 80)
fontInfo2 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf', 50)

for i in range(Chromosome.count):
    exec(f'drawObj.text((c{i}.textx, c{i}.texty), c{i}.location, fill="black", font=fontInfo2)')
    exec(f'''
ladder = 0
step = 80
for gene in c{i}.genes:
    if c{i}.texty == initialpoint:
        drawObj.text((c{i}.textx, 50), c{i}.species, fill="black",font=fontInfo1)
    ladder += step
    if "CACNG" in gene:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="red", font=fontInfo2)
    elif "TMEM114" in gene:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="red", font=fontInfo2)
    elif "TMEM235" in gene:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="red", font=fontInfo2)
    else:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="black", font=fontInfo2)
    ''')
newImage.show()
# newImage.save("out.png")
"""

exec(draw)

In [67]:
scoredict

{'PGS1': 5,
 'LOC103277795': 1,
 'SOCS3': 6,
 'LOC103277796': 1,
 'LOC100564571': 1,
 'TMEM235': 100,
 'BIRC5': 4,
 'AFMID': 6,
 'TK1': 6,
 'SYNGR2': 5,
 'TMC8': 1,
 'MYH9': 2,
 'LOC103278846': 1,
 'TXN2': 3,
 'FOXRED2': 5,
 'EIF3D': 6,
 'CACNG2': 100,
 'IFT27': 10,
 'PVALB': 7,
 'NCF4': 7,
 'CSF2RB': 4,
 'TEX33': 2,
 'CTNNA2': 1,
 'LRRTM1': 1,
 'LOC103279066': 1,
 'LOC103279065': 1,
 'LOC103279069': 1,
 'CACNG5': 100,
 'CACNG4': 100,
 'CACNG1': 100,
 'HELZ': 10,
 'PSMD12': 10,
 'PITPNC1': 6,
 'LOC107983219': 1,
 'NOL11': 4,
 'LOC103279756': 1,
 'RBFOX1': 7,
 'MIR5411': 1,
 'LOC107983238': 1,
 'LOC107983239': 1,
 'TMEM114': 100,
 'METTL22': 9,
 'ABAT': 9,
 'TMEM186': 6,
 'PMM2': 5,
 'CARHSP1': 5,
 'CCDC114': 1,
 'MYADM': 6,
 'LOC100557974': 1,
 'PRKCG': 7,
 'LOC103280215': 1,
 'CACNG6': 100,
 'CACNG8': 100,
 'CACNG7': 100,
 'LOC100560082': 1,
 'RPS9': 1,
 'LMTK3': 1,
 'LOC100560478': 1,
 'LOC100560675': 1,
 'AXIN2': 4,
 'CEP112': 6,
 'APOH': 4,
 'LOC103175619': 1,
 'TRNAA-GGC': 1,
 '(C

sort species

In [68]:
sortingspecies = [
    "Petromyzon marinus",
    "Callorhinchus milii", 
    "Danio rerio", 
    "Oreochromis niloticus", 
    "Xenopus tropicalis", 
    "Chrysemys picta", 
    "Anolis carolinensis", 
    "Canis lupus familiaris",
    "Mus musculus",
    "Homo sapiens", 
    "Gallus gallus"
    ]

sortedlist = []
tups = list(speciesdict.items())
tups[1][0]
for index,i in enumerate(sortingspecies):
    for index2, j in enumerate(tups):
        if j[0] == i:
            sortedlist.append(j)

speciesdict = dict(sortedlist)

Draw Image (to ppt)

In [69]:
from PIL import Image, ImageDraw, ImageFont

fontInfo1 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf', 60)
fontInfo2 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf', 50)

for index in range (Chromosome.count):

    exec(f"""newImage{index} = Image.new('RGBA', (850, 80*len(c{index}.genes)+250), 'white')
    
drawObj = ImageDraw.Draw(newImage{index})

ladder = 10

drawObj.text((10, ladder), c{index}.location, fill="black", font=fontInfo1)

ladder += 100

for gene in c{index}.genes:
    if "CACNG" in gene:
        drawObj.text((10, ladder), gene, fill="red", font=fontInfo2)
    elif "TMEM114" in gene:
        drawObj.text((10, ladder), gene, fill="red", font=fontInfo2)
    elif "TMEM235" in gene:
        drawObj.text((10, ladder), gene, fill="red", font=fontInfo2)
    elif scoredict[gene] >= 7:
        drawObj.text((10, ladder), gene, fill="blue", font=fontInfo2)
    else:

        drawObj.text((10, ladder), gene, fill="black", font=fontInfo2)
    ladder += 80

c{index}.image = newImage{index}

newImage{index}.save(r'C:\\Users\\junji\\Myworkspace\\Mylabwork\\data\\image\\C{index}.png')

""")

In [70]:
import pptx
from pptx.util import Inches, Pt

prs = pptx.Presentation()
prs.slide_width = Inches(11)
prs.slide_height = Inches(6)

slide_1 = prs.slides.add_slide(prs.slide_layouts[6])

lad1 = 0
for species in speciesdict:
    tb = slide_1.shapes.add_textbox(Inches(lad1-0.08),Inches(0),Inches(1),Inches(0.25))    #textbox
    tf = tb.text_frame
    p = tf.paragraphs[0]
    p.text = species
    p.font.size = Pt(7)
    lad2 = 0.25
    for c in speciesdict[species]:
        exec(f"tb = slide_1.shapes.add_picture(r'C:\\Users\\junji\\Myworkspace\\Mylabwork\\data\\image\\{c}.png',Inches(lad1),Inches(lad2),width=Inches(1))")
        exec(f"lad2 += (80*len({c}.genes)+250)/850")
    lad1+=1

prs.save(r"C:\Users\junji\Desktop\test.pptx")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\junji\\Desktop\\test.pptx'

In [ ]:
Chromosome.count

49